In [1]:
import pandas as pd
import json
import xmltodict

In [2]:
#Read Baidu Baike data
table=pd.read_csv('baiketriples/baike_triples.txt',delimiter='\t',usecols=['!='])

In [4]:
# The data are composed of triples of (entry, attribute name, attribute value)
# So, a certain entry could have multiple records in the table
# The scripts below are used to remove duplicates
table=table.drop_duplicates()
table.columns=['entity']

In [5]:
# Baidu Baike has 9M entries
table.shape

(9284909, 1)

In [ ]:
# Read Wiki index data
wiki=pd.read_csv('zhwiki/zhwiki-20230401-pages-articles-multistream-index.txt',delimiter=":",error_bad_lines=False)

In [44]:
# Rename Columns
wiki.columns=['offset','id','entity','title']

In [7]:
# Wiki China has 4M entries
wiki.shape

(4236515, 4)

In [8]:
# To covert traditional Chinese into simplified Chinese
import opencc
converter = opencc.OpenCC('t2s.json')

In [9]:
wiki['entity_sim']=wiki['title'].apply(lambda x: converter.convert(x) if type(x)==str else x)

In [102]:
# Keep the part of entity name before the '('
# e.g. 倩女幽魂 (1987年電影) → 倩女幽魂
wiki['entity_sim_clean']=wiki['entity_sim'].apply(lambda x: x.split('(')[0].strip() if type(x)==str else x)

In [103]:
# Keep the foreign figures' surnames
# e.g. 弗拉基米尔·列宁 → 列宁
wiki['entity_sim_clean']=wiki['entity_sim_clean'].apply(lambda x: x.split('·')[1].strip() if '·' in str(x) else x)

In [67]:
# Define a list of functional pages to be excluded
excludes=['Template', 'Category', 'Wikipedia', 'File', 'Topic', 'Portal',
       'MediaWiki', 'Module', 'WikiProject', 'Draft', 'Help', 'MOS', 'LTA']

In [104]:
# Get a list of entries that are not functional pages nor existing on Baike in its original form (direct comparison)
search_list=wiki[(~wiki['entity'].isin(excludes))&(~wiki['entity_sim_clean'].isin(table['entity']))]['entity_sim_clean'].unique()

In [105]:
# Exclude the overview pages, i.e. 列表 and 年表, which do not have equivalents on Baike because of disparities in UGC culture
search_list=[i for i in search_list if ('列表' not in i) and ('年表' not in i)]

In [106]:
# Find a total of 1.5M entries
len(search_list)

1505360

In [156]:
'六四事件' in search_list

True

In [158]:
'法轮功' in search_list

True

In [28]:
# Combine all Baike entities into a long string
bd_entities=table[~pd.isna(table['entity'])]['entity'].values
bd_entities='?'.join(bd_entities)

In [114]:
# Perform partial match
not_found_list=[i for i in search_list[:20000] if i not in bd_entities]

In [115]:
# 7975 of 20K Wiki entries are missing
len(not_found_list)

7975

In [42]:
# Read wiki content by offsets and lengths
# Since wiki content data is a super large file, we need to slice it to identify the useful part
# For example, if the index file suggests 六四事件 is located between 10000th and 20000th characters
# I would cut out this part and identify the entry related to 六四事件 
import bz2
import shutil

def decompress_chunk(wiki_filename, start_byte, data_length):
    temp_filename = 'temp.bz2'
    decomp_filename = 'temp.xml'

    with open(wiki_filename, 'rb') as wiki_file:
        wiki_file.seek(start_byte)
        data = wiki_file.read(data_length)

    with open(temp_filename, 'wb') as temp_file:
        temp_file.write(data)

    with bz2.BZ2File(temp_filename) as fr, open(decomp_filename,"wb") as fw:
        shutil.copyfileobj(fr,fw)

    return decomp_filename

In [46]:
indexes=list(wiki['offset'].unique())

In [116]:
wiki_lookup['length']=0
wiki_lookup=wiki[wiki['entity_sim_clean'].isin(not_found_list)].drop_duplicates('entity_sim_clean').copy()
current_lookup=wiki_lookup.copy()
while len(not_found_list)>0:
    redirects=[]
    lookup_indexes=current_lookup['offset'].unique()
    for i in lookup_indexes: #Many missing entries are located within the same offset range. I process them in one batch
        start=i
        length=indexes[indexes.index(i)+1]-i
        ids=current_lookup[current_lookup['offset']==start]['id'].values
        # Slice Wiki content
        decompress_chunk("zhwiki-20230401-pages-articles-multistream.xml.bz2",start,length)
        with open("temp.xml", encoding="utf8") as xml_file:
            data_dict = xmltodict.parse('<root>'+xml_file.read()+'</root>')
        json_data = json.dumps(data_dict)
        json_data = json.loads(json_data)
        # Go over every entry in the range and see if they are in the not_found_list
        for data in json_data['root']['page']:
            if int(data['id']) in ids:
                # Entries that require redirection would be handled in the next iteration
                if 'redirect' in data.keys():
                    if converter.convert(data['redirect']['@title'].split('(')[0].strip()) not in bd_entities:
                        redirects.append(data['redirect']['@title'])
                        print(data['title'],data['redirect']['@title']) #print out redirected cases
                # Evaluate the Wiki content length (in bytes)
                else:
                    wiki_lookup.loc[wiki_lookup['id']==int(data['id']),'length']=int(data['revision']['text']['@bytes'])
    #update the not_found_list = redirected entries that haven't been proceessed
    not_found_list=redirects
    current_lookup=wiki[wiki['title'].isin(redirects)].copy()
    wiki_lookup=pd.concat([wiki_lookup,current_lookup],axis=0)

# Keep entries with >10K bytes (≈ 3K Chinese characters)
wiki_lookup=wiki_lookup[wiki_lookup['length']>10000].reset_index(drop=True)

內容開放 開放內容
嚴重急性呼吸症候群 嚴重急性呼吸系統綜合症
台灣國旗 臺灣國旗
国际域名缩写 互联网顶级域列表
GNU通用公共协议证书 GNU通用公共许可证
同性恋和道德 宗教與同性戀
同性恋社区 LGBT社群
对同性恋的迫害 LGBT迫害
同性恋解放 LGBT權利運動
酷儿社区 LGBT社群
版权属左 Copyleft
8-4事件 2003年侵华日军遗弃在华化学毒剂泄漏事件
F sharp编程语言 F♯
九一一恐怖事件 九一一袭击事件
開爾文溫標 热力学温标
中国自然保护区名录 中华人民共和国国家级自然保护区列表
MSN Hotmail Outlook.com
美国历史时间表 美国历史年表
阿塞拜疆对外关系 阿塞拜疆外交
印度种姓 印度种姓制度
中国内战 中国战争列表
航空与航天工程 航空太空工程學
神舟號飛船 神舟载人飞船
公元前3世纪 前3世纪
香港行政区划 香港行政區劃
物理学定律一览表 物理学定律列表
HTML的字符編碼 HTML字符编码
威玛拼音 威妥瑪拼音
威码拼音 威妥瑪拼音
威妥玛拼法 威妥瑪拼音
威氏拼音 威妥瑪拼音
威式拼音 威妥瑪拼音
六四天安门事件 六四事件
政治联盟条约 马斯特里赫特条约
货币联盟条约 马斯特里赫特条约
聶斯妥里教派 東方亞述教會
科技史上的今天 公历日期列表
婚配问题 任务分配问题
婚姻匹配问题 任务分配问题
若泽·巴洛佐 若泽·曼努埃尔·巴罗佐
壬辰卫国战争 萬曆朝鮮之役
九一一恐怖襲擊事件 九一一袭击事件
🌣 太陽符號
阿丽亚娜火箭 亞利安系列運載火箭
日本天皇名单 日本天皇列表
变种克雅二氏病 变异型克-雅二氏病
伪沙皇季米特里 伪德米特里一世
神根公约 申根公约
羅馬皇帝一覽表 羅馬皇帝列表
他加洛語 他加祿語
反共产主义 反共主義
文禄·庆长之役 萬曆朝鮮之役
古罗马时间表 古罗马年表
亨利·內斯特萊 亨利·内斯莱
Moore机 摩尔型有限状态机
DBm 分贝毫瓦
香港基本法第廿三條 香港特別行政區基本法第二十三條
Hurd内核 GNU Hurd
微软视窗历史 Microsoft Windows的历史
Virginia Woolf 弗吉尼亚·吴尔夫
上海历任市长 上海市市长列表
火灾安全科学与工程 国际消防安全科学协会
魔戒三部曲：王者再臨 指环王：王者归来
鸡奸法在美国 美國性悖軌法
同性恋打压 LGBT

2005年中关村反日游行 2005年中国反日示威活动
中国抗日烈士 中国抗日战争牺牲者列表
天大BBS 天津大学BBS
拔略姓 拔略氏
2005年反日游行 2005年中国反日示威活动
吳乞買 金太宗
法属玻利尼西亚 法屬玻里尼西亞
Vegan 纯素主义
東施蒂利亞 上施蒂利亞
西施蒂利亞 上施蒂利亞
麥拿輪車隊 麥拉倫車隊
日本棒球殿堂 野球殿堂博物館
鐵利亞 㹴犬
美国原住民 美國原住民
長途電話代碼 国际电话区号列表
誠泰Cobras 米迪亞暴龍
誠泰Cobras隊 米迪亞暴龍
越南陳朝世系 越南陳朝及胡朝君主世系圖
孛儿只斤爱猷识理答腊 元昭宗
日本国立大学 国立大学法人
市立天母國小 臺北市士林區天母國民小學
風土病 地方性流行病
中日韩统一表意文字 中日韓統一表意文字
統一漢字編碼 中日韓統一表意文字
2005年中國反日遊行 2005年中国反日示威活动
九歲的人生 童行歲月
動畫特別詞語 動畫術語列表
白遼士 埃克托·柏辽兹
梅西耶星表 梅西耶天体列表
巨鷹珍寶 丹拿／利蘭珍寶巴士
香港地方 香港地方列表
C&S 動畫術語列表
香港泳灘 香港海灘
即时通讯软件的比较 即时通讯软件比较
海爾博普彗星 海爾-博普彗星
克希荷夫電流定律 基爾霍夫電路定律
克希荷夫電壓定律 基爾霍夫電路定律
极光奏鸣曲 極光奏鳴曲樂團
太阳及日光层探测仪 太陽和太陽圈探測器
支出最小化问题 支出最小化
CHOBITS Chobits
第四世夏坝活佛 夏坝·格勒克珠·降白降措
淺井達子 崇源院
海爾博普 海爾-博普彗星
柯爾莫哥洛夫公理 機率公設
The Wiki Way Wiki之道
GNOME Web GNOME 網頁
承德外八庙 承德避暑山庄及其周围寺庙
QuakeIII 雷神之锤III竞技场
誠泰眼鏡蛇隊 米迪亞暴龍
四號幹線 香港4號幹線
六號幹線 香港6號幹線
八號幹線 香港8號幹線
戰慄時空 半衰期系列
香港道路網路 香港道路
An-225 安托諾夫安-225運輸機
秦末農民戰爭 秦末民变
興農牛隊 富邦悍將
興農熊 富邦悍將
興農熊隊 富邦悍將
洛歇三星 洛歇L-1011
公民及政治權利國際公約 公民权利及政治权利国际公约
西夏献宗 夏献宗
GNU General Public License GNU通用公共许可证
La new熊隊 樂天桃猿
西澤祐司 全日空61號

In [148]:
wiki_lookup.sort_values('length',ascending=False)

,offset,id,entity,title,entity_sim,entity_sim_clean,length
80,25958007,9066,联合国教科文组织非物质文化遗产名录和优秀保护实践名册,NaN,联合国教科文组织非物质文化遗产名录和优秀保护实践名册,联合国教科文组织非物质文化遗产名录和优秀保护实践名册,289402.0
85,26645569,9512,1956年匈牙利革命,NaN,1956年匈牙利革命,1956年匈牙利革命,271865.0
62,20871516,6503,马哈迪·莫哈末,NaN,马哈迪·莫哈末,莫哈末,229924.0
135,43252334,19079,釣魚臺列嶼主權問題,NaN,钓鱼台列屿主权问题,钓鱼台列屿主权问题,223193.0
801,143657679,103045,重性抑鬱疾患,NaN,重性抑郁疾患,重性抑郁疾患,205888.0
...,...,...,...,...,...,...,...
1008,164683918,121588,詹遜,NaN,詹逊,詹逊,10016.0
587,115085321,76828,1997年德国大奖赛,NaN,1997年德国大奖赛,1997年德国大奖赛,10013.0
838,148564820,106836,英特爾國際科技展覽會,NaN,英特尔国际科技展览会,英特尔国际科技展览会,10008.0
526,106814697,70097,GUNDAMonline,NaN,GUNDAMonline,GUNDAMonline,10003.0


In [147]:
wiki_lookup.sort_values('length',ascending=False)[['entity_sim_clean','length']].to_csv('unmatched_entities2.csv',index=False)

In [142]:
wiki[wiki['entity_sim_clean'].apply(lambda x: '六四' in str(x))]

,offset,id,entity,title,entity_sim,entity_sim_clean
2631,21366537,6671,六四天安门事件,NaN,六四天安门事件,六四天安门事件
7385,37870755,16288,六四天安門事件,NaN,六四天安门事件,六四天安门事件
7567,38892804,16773,六四,NaN,六四,六四
40699,127566482,89038,六四学潮,NaN,六四学潮,六四学潮
80911,217553222,179998,六四运动,NaN,六四运动,六四运动
...,...,...,...,...,...,...
4027114,2641850480,7991503,六四事件三十三週年,NaN,六四事件三十三周年,六四事件三十三周年
4027568,2642105499,7992218,六四酒案,NaN,六四酒案,六四酒案
4030897,2644093685,7997700,六四事件三十三周年,NaN,六四事件三十三周年,六四事件三十三周年
4031915,2644645427,7999223,八酒六四,NaN,八酒六四,八酒六四


In [159]:
table[table['entity'].apply(lambda x: '六四' in str(x))]

,entity
3401815,一八六四之力挽狂澜
3402208,一六四0年英国革命史
3402209,一六四四
3402219,一六四零年英国革命史
5704638,中国人民解放军第二六四医院
6354267,中棉所六四
10413053,公元一六四四
10498513,六四寨村
10498522,六四庄
10498527,六四班的追星族
